# Seminário: Sistema Baseado em Conhecimento 

**Disciplina**: Inteligência Artificial  
**Projeto**: Sistema de Recomendação de Itens e Magias para Enfrentar Inimigos  
**Alunos**: Gabryel Camillo Leite, Murilo Aldigueri Marino

---

## Introdução

Neste seminário, vamos explorar a implementação de um **Sistema Baseado em Conhecimento** para recomendar itens e magias adequados contra inimigos, utilizando o **algoritmo Rete** como motor de inferência. O sistema avalia as características dos inimigos e recomenda as melhores estratégias, otimizando o processo de decisão. Vamos abordar os conceitos teóricos por trás do algoritmo Rete e mostrar como ele foi aplicado neste projeto.

---

## Fundamentos do Algoritmo Rete

O **algoritmo Rete** é amplamente utilizado em **Sistemas Baseados em Conhecimento** e foi projetado para melhorar a eficiência de motores de inferência. Ele funciona armazenando e reutilizando subexpressões comuns entre regras, reduzindo a necessidade de repetição de verificações a cada ciclo de inferência.

- **Regras de Produção**: Conjunto de regras que descrevem ações a serem executadas com base em determinadas condições.
- **Estrutura em Grafo**: O Rete constrói um grafo para representar as regras, otimizando a combinação de padrões.
- **Compartilhamento de Subexpressões**: O motor compartilha partes de regras que se repetem, tornando a inferência mais eficiente.

## Otimização no Código

Implementação um motor de inferência básico, mas não há um grafo de regras ou de compartilhamento de subexpressões como acontece no algoritmo Rete clássico. 

### 1. Uso de Listas e Iteração Linear:

- O código percorre todas as regras e avalia itens e magias em relação a elas, acumulando pontuações. Este processo é eficiente para um número razoável de regras, dado que você não repete regras já avaliadas para os mesmos itens.

### 2. Classificação Após Avaliação:

- Após a execução do motor de inferência, os itens/magias são classificados por pontuação, o que permite a exibição apenas dos mais relevantes. Isso garante que a interface apresente resultados de maneira eficiente e evita verificações repetidas na interface do usuário.

### 3. Verificações Condicionais:

- O código só avalia regras que são diretamente aplicáveis aos atributos dos itens/magias (como damage_types ou conditions_inflicted). Ao evitar verificações desnecessárias (como avaliar regras de imunidade a dano em itens que não infligem dano).

---

## Estrutura da Implementação

A implementação é composta por várias classes e um motor de inferência:

### 1. Classe `Enemy`

```python
class Enemy:
    def __init__(self, name, vulnerabilities, resistances, immunities, condition_immunities):
        self.name = name
        self.vulnerabilities = vulnerabilities
        self.resistances = resistances
        self.immunities = immunities
        self.condition_immunities = condition_immunities

```
- **Explicação:** A classe `Enemy` modela um inimigo, com atributos como vulnerabilidades, resistências, imunidades e imunidades a condições.
- **Uso:** Estas informações serão usadas pelo motor de inferência para calcular a eficácia dos itens/magias recomendados.

### 2. Classe `Item`

```python
class Item:
    def __init__(self, name, damage_types):
        self.name = name
        self.damage_types = damage_types

```
- **Explicação:** A classe `Item` representa itens com diferentes tipos de dano. Estes serão comparados às vulnerabilidades e resistências dos inimigos.

### 3. Classe `Spell`

```python
class Spell:
    def __init__(self, name, damage_types, conditions):
        self.name = name
        self.damage_types = damage_types
        self.conditions = conditions

```
- **Explicação:** Similar à classe `Item`, mas para magias, que também têm condições especiais que podem infligir efeitos no inimigo.

### 4. Classe `Rule`

```python
class Rule:
    def __init__(self, enemy_attribute, effect_type, weight):
        self.enemy_attribute = enemy_attribute 
        self.effect_type = effect_type 
        self.weight = weight 

    def matches(self, item: Union[Item, Spell]):
        # Verifica se a propriedade do item corresponde ao atributo do inimigo para a regra
        return self.enemy_attribute in item.properties

```
- **Explicação:** A classe `Rule`, define uma regra de produção, associando um atributo do inimigo (por exemplo, uma vulnerabilidade) ao atributo de um item, e também a um peso que será usado para calcular a pontuação do item conforme a regra.

### Motor de Inferência: `InferenceEngine`

```python
class Inference:
    def __init__(self, rules):
        self.rules = rules

    # Função para Avaliação de Itens e Magias
    def match(self, items: List[Union[Item, Spell]]):
        matched_items = []
        for item in items:
            score = 0
            for rule in self.rules:
                if rule.matches(item):
                    score += rule.weight

            # Se o item inflige condições, aumenta a pontuação
            if hasattr(item, "conditions_inflicted"):
                for _ in item.conditions_inflicted:
                    score += 1  # Proporcional a imunidade a condições

            if score > 0:
                matched_items.append((item, score))

        return matched_items

    def run(self, items: List[Union[Item, Spell]]):
        applicable_items = self.match(items)
        applicable_items.sort(key=lambda x: x[1], reverse=True)
        return applicable_items  # Retorna a lista ordenada de tuplas (item, score)
        
```
- **Explicação:**  O `InferenceEngine` é responsável por processar as regras e fazer a correspondência entre os atributos dos inimigos e os itens/magias disponíveis. 

O código que implementa o motor de regras (ReteEngine) no exemplo é progressivo, não recursivo. 
O processo de avaliação de itens segue um fluxo linear, onde:
1. O método run chama o método match para avaliar todos os itens, um por um.
2. Cada item é comparado com o inimigo, utilizando as regras fornecidas.
3. Para cada regra, é feita uma simples verificação condicional (por exemplo, se o item corresponde a uma vulnerabilidade, resistência ou imunidade).
4. Se algum item corresponde, a pontuação é ajustada e o item é adicionado à lista de correspondências.

##  File `ia.py`

### 1. Visão Geral

O arquivo ia.py utiliza uma abordagem baseada em sistemas de recomendação e lógica para criar um sistema de recomendação de itens e magias adequados contra inimigos, levando em consideração suas vulnerabilidades, resistências e imunidades. Para isso, ele carrega informações de arquivos JSON, cria objetos de classes customizadas (como Enemy, Item, Spell) e utiliza o algoritmo Rete para processar regras de inferência.

### 2. Funções de Manipulação de Dados

### 2.1 `load_data()`

Carrega os dados de arquivos JSON:

```python
def load_data(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

```
Essa função lê um arquivo JSON e retorna um dicionário Python contendo os dados para manipulação futura.

### 2.2 `create_enemies()`, `create_items()`, `create_spells()`

Essas funções criam instâncias das classes `Enemy`, `Item` e `Spell` com base nos dados carregados dos arquivos JSON:

```python
def create_enemies(data):
    enemies = []
    for enemy_data in data:
        enemy = Enemy(
            enemy_data["Name"],
            enemy_data["Damage Vulnerabilities"],
            enemy_data["Damage Resistances"],
            enemy_data["Damage Immunities"],
            enemy_data["Condition Immunities"],
        )
        enemies.append(enemy)
    return enemies

```
Cada uma dessas funções percorre a lista de dados e cria objetos correspondentes. O processo é similar para itens e magias, utilizando os atributos apropriados.

### 3. Criação de Regras com Base nos Atributos do Inimigo

### 3.1 `create_rules()`

Esta função é responsável por criar as regras de correspondência entre os atributos do inimigo (como vulnerabilidades e resistências) e as propriedades dos itens e magias:

```python
def create_rules(enemy):
    rules = []
    # Peso é baseado no tipo de efeito da regra
    # Criar regras para vulnerabilidades
    for vulnerability in enemy.vulnerabilities:
        rules.append(
            Rule(
                enemy_attribute=vulnerability,
                effect_type=RuleType.VULNERABILITY,
                weight=3,  # Vulnerabilidade aumenta a pontuação
            )
        )
    # Similar para resistências, imunidades e condições
    return rules

```
Essas regras são fundamentais para o motor de inferência, já que elas permitem identificar quais itens e magias são eficazes contra um determinado inimigo.

### 4. Funções para Exibir Recomendações

### 4.1 `display_recommended_items()` e `display_recommended_spells()`

Essas funções exibem uma lista de itens ou magias recomendados com base nas regras criadas. Elas implementam paginação e navegação no terminal, utilizando a biblioteca `curses`:

```python
def display_recommended_items(stdscr, selected_enemy, items):
    ITEMS_PER_PAGE = 10
    current_row = 0
    page = 0
    rules = create_rules(selected_enemy)
    rete_engine = ReteEngine(rules)
    recommendations_with_score = rete_engine.run(selected_enemy, items)
    TOTAL_PAGES = (len(recommendations_with_score) - 1) // ITEMS_PER_PAGE + 1

    # Código para navegação e exibição dos itens recomendados...

```
Essas funções utilizam o motor Rete para calcular as pontuações dos itens ou magias recomendados com base nas regras definidas e permitem ao usuário navegar pelas recomendações no terminal.

### 5. Navegação entre Inimigos e Tipos de Recomendação

### 5.1 `choose_recommendation_type()`

Permite ao usuário escolher entre itens ou magias como recomendação, com navegação via setas no terminal:

```python
def choose_recommendation_type(stdscr, selected_enemy, items, spells):
    options = ["Itens", "Magias"]
    # Código de navegação para selecionar o tipo de recomendação...

```

### 5.2 `curses_menu()`

Cria o menu inicial onde o usuário seleciona o inimigo para o qual deseja obter recomendações:

```python
def curses_menu(stdscr, enemies, items, spells):
    ITEMS_PER_PAGE = 10
    TOTAL_PAGES = (len(enemies) - 1) // ITEMS_PER_PAGE + 1
    current_row = 0
    page = 0

    # Exibe uma lista de inimigos e permite a navegação para escolher um...

```

### 5.3 `main()`

A função principal que inicializa o programa, carregando os dados e iniciando o menu:

```python
def main(stdscr):
    curses.start_color()
    curses.init_pair(1, curses.COLOR_GREEN, curses.COLOR_BLACK)
    enemies_data = load_data("data/enemies.json")
    items_data = load_data("data/items.json")
    spells_data = load_data("data/spells.json")
    enemies = create_enemies(enemies_data)
    items = create_items(items_data)
    spells = create_spells(spells_data)
    curses_menu(stdscr, enemies, items, spells)

```

## Exemplo Prático

Demonstração de como o sistema funciona:

### 1. Definindo Inimigo

In [1]:
from classes.enemy import Enemy

enemy = Enemy(
    name="Dragon",
    vulnerabilities=["ice", "water"],
    resistances=["fire", "earth"],
    immunities=["poison"],
    condition_immunities=["stun"]
)

- **Explicação:** O inimigo `Dragon` é vulnerável a gelo e água, resistente a fogo e terra, e imune a veneno. Além disso, ele é imune à condição atordoar.

### 2. Avaliação de Itens

In [ ]:
from classes.item import Item
from classes.inference_engine import InferenceEngine
from classes.rule import Rule, RuleType

# Exemplo de itens
item1 = Item(name="Ice Sword", properties=["ice"])
item2 = Item(name="Fire Axe", properties=["fire"])
item3 = Item(name="Earth-Ice Dagger", properties=["earth", "ice"])

# Criando as regras a partir das propriedades do inimigo
rules = [
    Rule(enemy_attribute="ice", effect_type=RuleType.VULNERABILITY, weight=3),
    Rule(enemy_attribute="water", effect_type=RuleType.VULNERABILITY, weight=3),
    Rule(enemy_attribute="fire", effect_type=RuleType.RESISTANCE, weight=-2),
    Rule(enemy_attribute="earth", effect_type=RuleType.RESISTANCE, weight=-2),
    Rule(enemy_attribute="poison", effect_type=RuleType.IMMUNITY, weight=-3),
]

# Criando o motor de inferência
engine = InferenceEngine(rules)

# Executando o motor com o inimigo e os itens
items = [item1, item2, item3]
scores = engine.run(items)

# Exibir os itens recomendados com as pontuações
for item, score in scores:
    print(f"Item: {item.name}, Score: {score}")


**Saída Esperada:**

{'Ice Sword': 3, 'Earth-Ice Dagger': 1, 'Fire Axe': -2}

- **Explicação:** A `Ice Sword` recebe 3 de pontuação devido à vulnerabilidade do Dragão a gelo, enquanto a `Earth-Ice Dagger` recebe 1 de pontuação devido a resistência a terra(-2) e vulnerabilidade a gelo(+3), já a `Fire Axe` perde pontos devido à resistência do inimigo ao fogo (Como sua pontuação não é positiva ela não aparece nas recomendações).